In [23]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# تعريف المتغيرات خارج الحلقة
counter = 0
stage = None

# فتح الكاميرا
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # تحويل الصورة إلى RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # معالجة الصورة باستخدام Mediapipe
        results = pose.process(image)
        
        # تحويل الصورة إلى BGR للعرض باستخدام OpenCV
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # حساب الزوايا
        def calculate_angle(a, b, c):
            a = np.array(a)
            b = np.array(b)
            c = np.array(c)
            
            radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
            angle = np.abs(radians * 180.0 / np.pi)
            
            if angle > 180.0:
                angle = 360 - angle
            
            return angle
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # استخراج إحداثيات المفاصل
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # حساب زاوية الكوع
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # عرض الزاوية على الصورة
            cv2.putText(image, str(angle), tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # منطق العد
            if angle > 130:
                stage = "down"
            if angle < 60 and stage == "down":
                stage = 'up'
                counter += 1
                print(counter)
        
        except:
            pass
        
        # عرض العد والمرحلة
        cv2.rectangle(image, (0, 0), (235, 73), (245, 117, 16), -1)
        
        cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1, cv2.LINE_AA)
        
        cv2.putText(image, 'STAGE', (75, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (80, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1, cv2.LINE_AA)
        
        # رسم نقاط الهيكل العظمي
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(145, 117, 66), thickness=4),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=4))
        
        # عرض الصورة
        cv2.imshow('Mediapipe Feed', image)
        
        # إنهاء العرض عند الضغط على 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# تحرير الكاميرا وإغلاق النوافذ
cap.release()
cv2.destroyAllWindows()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [24]:
results.pose_landmarks


landmark {
  x: 0.689359128
  y: 0.708578169
  z: -2.12651491
  visibility: 0.996694386
}
landmark {
  x: 0.738645077
  y: 0.594361484
  z: -2.06808615
  visibility: 0.994104
}
landmark {
  x: 0.76518774
  y: 0.593351305
  z: -2.06864715
  visibility: 0.995173097
}
landmark {
  x: 0.793471873
  y: 0.593124926
  z: -2.06907606
  visibility: 0.993254244
}
landmark {
  x: 0.641081214
  y: 0.586105406
  z: -2.05698061
  visibility: 0.995071769
}
landmark {
  x: 0.609776378
  y: 0.58074832
  z: -2.05625963
  visibility: 0.99629277
}
landmark {
  x: 0.583451509
  y: 0.576935887
  z: -2.05696225
  visibility: 0.996171892
}
landmark {
  x: 0.837443531
  y: 0.594653547
  z: -1.42784703
  visibility: 0.995004356
}
landmark {
  x: 0.551781118
  y: 0.568321288
  z: -1.32995868
  visibility: 0.997859597
}
landmark {
  x: 0.742336392
  y: 0.797605515
  z: -1.85886633
  visibility: 0.997059107
}
landmark {
  x: 0.630885243
  y: 0.798529804
  z: -1.83540881
  visibility: 0.998254478
}
landmark {
  x: 

الحمد لله رب العالمين  هذا من فضل ربي